In [ ]:
!pip install --upgrade transformers scipy ftfy accelerate diffusers

Cloning into 'adversarial_attack_on_SD'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 110 (delta 49), reused 87 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (110/110), 13.85 KiB | 3.46 MiB/s, done.
Resolving deltas: 100% (49/49), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━

In [3]:
import os
import json
# Folder for store 
os.mkdir("log/sentence")

# Load sentence 
with open('dataset/sentence20.json', 'r') as file:
    data = json.load(file)
# Copy 
with open('log/sentence/sentence20.json', 'w') as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

oo = data['oo']
to = data['to']
ori_sentence = data['ori_sentence'] 
tar_sentence = data['tar_sentence'] 
print('Original object:', oo)
print('Target object:', to)
print('Original sentence:', ori_sentence)
print('Target sentence:', tar_sentence)


Original object: basket
Target object: banana
Original sentence: a basket
Target sentence: a basket and a banana


In [4]:
import torch
from transformers import CLIPTextModel, CLIPTokenizer

len_prompt = 5

tokenizer = CLIPTokenizer.from_pretrained('openai/clip-vit-large-patch14')
text_encoder = CLIPTextModel.from_pretrained('openai/clip-vit-large-patch14')
text_encoder = text_encoder.to('cuda')

2025-05-15 05:19:08.428915: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747286348.618899      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747286348.676671      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

### Avoidance strategy

In [5]:
from src import *
char_list = get_char_table()
print(len(char_list))
char_list = avoidance_strategy(char_list,to, tokenizer, text_encoder)
print(len(char_list))

61
word: banana min_word: a min_score: 0.6257330179214478
60


# NoTE

In [6]:
os.makedirs("log/sentence/NoTE") 
cosin = compare_sentences(tar_sentence,ori_sentence,mask=None,tokenizer = tokenizer ,text_encoder = text_encoder) 
print(cosin) 
with open('log/sentence/NoTE/cosin.json', 'w', encoding='utf-8') as f:
    json.dump(cosin, f, ensure_ascii=False, indent=4)

0.780128538608551


## Beam Uniform

In [7]:
os.makedirs("log/sentence/NoTE/beam_uni")
beam_widths = [60,140,140,140,3]
score_dict, pool_score_log = beam_search(tar_sentence, ori_sentence, char_list, len_prompt, None, tokenizer, text_encoder,beam_widths)
# Save
with open('log/sentence/NoTE/beam_uni/score_dict.json', 'w', encoding='utf-8') as f:
    json.dump(score_dict, f, ensure_ascii=False, indent=4)
with open('log/sentence/NoTE/beam_uni/pool_score_log.json', 'w', encoding='utf-8') as f:
    json.dump(pool_score_log, f, ensure_ascii=False, indent=4)


## Beam Increasing

In [8]:
os.makedirs("log/sentence/NoTE/beam_in")
beam_widths = [60,20,50,350,3]
score_dict, pool_score_log = beam_search(tar_sentence, ori_sentence, char_list, len_prompt, None, tokenizer, text_encoder,beam_widths)
# Save
with open('log/sentence/NoTE/beam_in/score_dict.json', 'w', encoding='utf-8') as f:
    json.dump(score_dict, f, ensure_ascii=False, indent=4)
with open('log/sentence/NoTE/beam_in/pool_score_log.json', 'w', encoding='utf-8') as f:
    json.dump(pool_score_log, f, ensure_ascii=False, indent=4)


## Beam Decreasing

In [9]:
os.makedirs("log/sentence/NoTE/beam_de")
beam_widths = [60,350,50,20,3]
score_dict, pool_score_log = beam_search(tar_sentence, ori_sentence, char_list, len_prompt, None, tokenizer, text_encoder,beam_widths)
# Save
with open('log/sentence/NoTE/beam_de/score_dict.json', 'w', encoding='utf-8') as f:
    json.dump(score_dict, f, ensure_ascii=False, indent=4)
with open('log/sentence/NoTE/beam_de/pool_score_log.json', 'w', encoding='utf-8') as f:
    json.dump(pool_score_log, f, ensure_ascii=False, indent=4)


## POPOP

In [10]:
os.makedirs("log/sentence/NoTE/popop")
generation_num = 50 
generation_scale = 1280
tour_size = 4
seeds = np.arange(22520465,22520465+10)
for seed in seeds:
    seed = int(seed)
    os.makedirs(f"log/sentence/NoTE/popop/{seed}")
    score_dict, pool_score_log = evolution_strategy(tar_sentence, ori_sentence, char_list, len_prompt, generation_num, generation_scale, tokenizer, text_encoder, tour_size, mask = None,seed=seed)
    # Save
    with open(f'log/sentence/NoTE/popop/{seed}/score_dict.json', 'w', encoding='utf-8') as f:
        json.dump(score_dict, f, ensure_ascii=False, indent=4)
    with open(f'log/sentence/NoTE/popop/{seed}/pool_score_log.json', 'w', encoding='utf-8') as f:
        json.dump(pool_score_log, f, ensure_ascii=False, indent=4)


# TE

In [11]:
os.makedirs("log/sentence/TE")
te, cosin = auto_tune_threshold(tar_sentence,ori_sentence,tokenizer,text_encoder,start = 1,end = 10, space_limit = 2000)
with open('log/sentence/TE/cosin.json', 'w', encoding='utf-8') as f:
    json.dump(cosin, f, ensure_ascii=False, indent=4)


## Beam Uniform

In [12]:
os.makedirs("log/sentence/TE/beam_uni")
beam_widths = [60,140,140,140,3]
score_dict, pool_score_log = beam_search(tar_sentence, ori_sentence, char_list, len_prompt, te, tokenizer, text_encoder,beam_widths)
# Save
with open('log/sentence/TE/beam_uni/score_dict.json', 'w', encoding='utf-8') as f:
    json.dump(score_dict, f, ensure_ascii=False, indent=4)
with open('log/sentence/TE/beam_uni/pool_score_log.json', 'w', encoding='utf-8') as f:
    json.dump(pool_score_log, f, ensure_ascii=False, indent=4)


## Beam Increasing

In [13]:
os.makedirs("log/sentence/TE/beam_in")
beam_widths = [60,20,50,350,3]
score_dict, pool_score_log = beam_search(tar_sentence, ori_sentence, char_list, len_prompt, te, tokenizer, text_encoder,beam_widths)
# Save
with open('log/sentence/TE/beam_in/score_dict.json', 'w', encoding='utf-8') as f:
    json.dump(score_dict, f, ensure_ascii=False, indent=4)
with open('log/sentence/TE/beam_in/pool_score_log.json', 'w', encoding='utf-8') as f:
    json.dump(pool_score_log, f, ensure_ascii=False, indent=4)


## Beam Decreasing

In [14]:
os.makedirs("log/sentence/TE/beam_de")
beam_widths = [60,350,50,20,3]
score_dict, pool_score_log = beam_search(tar_sentence, ori_sentence, char_list, len_prompt, te, tokenizer, text_encoder,beam_widths)
# Save
with open('log/sentence/TE/beam_de/score_dict.json', 'w', encoding='utf-8') as f:
    json.dump(score_dict, f, ensure_ascii=False, indent=4)
with open('log/sentence/TE/beam_de/pool_score_log.json', 'w', encoding='utf-8') as f:
    json.dump(pool_score_log, f, ensure_ascii=False, indent=4)


## POPOP

In [15]:
os.makedirs("log/sentence/TE/popop")
generation_num = 50 
generation_scale = 1280
tour_size = 4
seeds = np.arange(22520465,22520465+10)
for seed in seeds: 
    seed = int(seed)
    os.makedirs(f"log/sentence/TE/popop/{seed}")
    score_dict, pool_score_log = evolution_strategy(tar_sentence, ori_sentence, char_list, len_prompt, generation_num, generation_scale, tokenizer, text_encoder, tour_size, mask = te,seed=seed)
    # Save
    with open(f'log/sentence/TE/popop/{seed}/score_dict.json', 'w', encoding='utf-8') as f:
        json.dump(score_dict, f, ensure_ascii=False, indent=4)
    with open(f'log/sentence/TE/popop/{seed}/pool_score_log.json', 'w', encoding='utf-8') as f:
        json.dump(pool_score_log, f, ensure_ascii=False, indent=4)
